# 02. 다변량 정규분포와 수치 선형대수 (Multivariate normal distributions and numerical linear algebra)

- 다변수 정규분포는 통계학에서 가장 일반적으로 접하는 분포 중 하나
- $d$차원의 다변량 정규분포 $N(\mu, \Sigma)$는

  $d$차원의 평균벡터 $\mu$와, $d \times d$ 차원의 양의 정부호 대칭 공분산 행렬 $\Sigma$를 매개변수로 가짐

$$\mathcal{N}(\mu, \Sigma) = \dfrac{1}{(2\pi)^{d/2} |\Sigma| ^{1/2}} \exp \left( -\dfrac{1}{2} (x-\mu)^T \Sigma^{-1} (x-\mu) \right)$$

- $\Sigma$의 속성
  - 대칭행렬의 모든 고유값은 실수
  - 모든 양의 정부호 대칭행렬의 고유값은 양수

<br>

## 02-01. 촐레스키 분해 (Cholesky Decomposition)
- 대칭 양의 정부호 행렬 $\Sigma$는 $\mathbf{\Sigma} = \mathbf{L} \mathbf{L}^T,$로 표현 가능
  - $L$은 $\Sigma$의 촐레스키 분해라고 불리는 하삼각행렬

<br>

- 참고: Python `NumPy`와 `SciPy`는 촐레스키 분해의 두 가지 호환되지 않는 구현을 가짐
  - `numpy.linalg.cholesky`는 위의 정의와 동일한 방식을 사용
  - `scipy.linalg.cholesky`는 상삼각행렬 $U=L^T$를 `default`로 반환

<br>

## 02-02. 다변량 정규분포 평가
- $d$차원의 다변량 정규분포의 로그 밀도함수는

$$\log p(\mathbf{x}) = -\frac{d}{2} \log(2 \pi) -\frac{1}{2} \log | \det \mathbf{\Sigma} | - \frac{1}{2} (\mathbf{x}-\mathbf{\mu})^T \mathbf{\Sigma}^{-1} (\mathbf{x}-\mathbf{\mu}).$$

- $\log | \det \mathbf{\Sigma} |$과, $- \frac{1}{2} (\mathbf{x}-\mathbf{\mu})^T \mathbf{\Sigma}^{-1} (\mathbf{x}-\mathbf{\mu}).$는 촐레스키 분해를 사용하여 효율적으로 수치적으로 안정적인 방식으로 평가 가능

<br>

### 01. 촐레스키 분해를 통한 행렬식 평가

- $\Sigma = LL^T$이고, $L = \begin{pmatrix}  l_{11} & 0 & 0 & \dots & 0 \\  l_{21} & l_{22} & 0 & \dots & 0 \\  l_{31} & l_{32} & l_{33} & \dots & 0 \\  \vdots & \vdots & \vdots & \ddots & 0 \\  l_{d1} & l_{d2} & l_{d3} & \dots & l_{dd} \\ \end{pmatrix}$일 때

  행렬식과 로그의 기본 속성을 사용하여,

$$\begin{align*}
  \log \det \boldsymbol{\Sigma} &= \log\left( \det ( \mathbf{L} \mathbf{L}^T) \right)
    = \log\left( \det \mathbf{L} \det \mathbf{L}^T \right) \\
    &= \log\left( (\det \mathbf{L})^2 \right)
    = 2 \log\left( \det \mathbf{L} \right) \\
    &= 2 \log\left( \prod_{i=1}^d l_{ii} \right)
    = 2 \sum_{i=1}^d \log\left( l_{ii} \right).
\end{align*}$$

<br>

### 02. 촐레스키를 사용한 2차 형식 평가
- 2차 형식을 다음과 같이 단순화

$$\begin{align*}
    (\mathbf{x}-\boldsymbol{\mu})^T \boldsymbol{\Sigma}^{-1} (\mathbf{x}-\boldsymbol{\mu})
    &= (\mathbf{x}-\boldsymbol{\mu})^T (\mathbf{L} \mathbf{L}^T)^{-1} (\mathbf{x}-\boldsymbol{\mu}) \\
    &= (\mathbf{x}-\boldsymbol{\mu})^T \mathbf{L}^{-T} \mathbf{L}^{-1} (\mathbf{x}-\boldsymbol{\mu}) \\
    &= (\mathbf{L}^{-1}(\mathbf{x}-\boldsymbol{\mu}))^T \mathbf{L}^{-1}(\mathbf{x}-\boldsymbol{\mu}) \\
	&= \mathbf{z}^T \mathbf{z} = \sum_{i=1}^d z_i^2,
\end{align*}$$

$$where \; \mathbf{z} = (z_1, \dots, z_d)^T = \mathbf{L}^{-1}(\mathbf{x}-\mathbf{\mu})$$

- $z=L^{-1}(x- \mu)$를 계산하기 위하여, $\mathbf{L} \mathbf{z} = \mathbf{x}-\mathbf{\mu}$의 선형식의 해를 계산

  $L$은 삼각행렬이기에

```python
import scipy.linalg as slg
z = slg.solve_triangular(L, x-mu, lower=True)
```


- 일반적으로 양의 정부호 행렬인 $\Sigma$에 대해, $\Sigma x = b$의 해를 구하는 최고의 방법은,
  
  $\Sigma$에 대한 내부적 촐레스키 분해를 사용하는 것

<br>

## 02-03. 다변량 정규분포의 변환

<br>

#### Theorem
- $x \sim N(0, I_d)$일 때, $y-N(\mu, \Sigma)$는 $y=Lx + \mu$변환을 통해 얻을 수 있음
  - $I_d$는 단위 공분산 행렬
  - $L$은 $\Sigma$의 촐레스키 분해 $\Sigma=LL^T$ 요소

- $y$의 평균과 공분산 행렬은

$$\operatorname{E}[\mathbf{y}] = \mathbf{L} \operatorname{E}[\mathbf{x}] + \mathbf{\mu} = \mathbf{\mu}$$

$$\operatorname{Cov}[\mathbf{y}] = \operatorname{E}[ (\mathbf{y} - \operatorname{E}[\mathbf{y}]) (\mathbf{y} - \operatorname{E}[\mathbf{y}])^T ]
= \operatorname{E}[ (\mathbf{L} \mathbf{x})(\mathbf{L} \mathbf{x})^T]
= \operatorname{E}[ \mathbf{L} \mathbf{x} \mathbf{x}^T \mathbf{L}^T ]
= \mathbf{L} \operatorname{E}[ \mathbf{x} \mathbf{x}^T ] \mathbf{L}^T
= \mathbf{L} \mathbf{I}_d \mathbf{L}^T = \mathbf{\Sigma}.$$

<br>

- $y$는 아핀 변환 후에도 다변량 정규 분포를 유지
  -  $x=Az+b$ 변환된 (비정규화된) 다변량 정규 분포는, 여전히 $z$에 대한 다변량 정규분포의 형태이기 때문

$$\begin{align*}
p(\mathbf{x}) &= \frac{1}{Z} \exp(- \frac{1}{2} (\mathbf{x}-\boldsymbol{\mu})^T \boldsymbol{\Sigma}^{-1} (\mathbf{x}-\boldsymbol{\mu}))\\
       &= \frac{1}{Z'} \exp(- \frac{1}{2} (\mathbf{A} \mathbf{z} + \mathbf{b} -\boldsymbol{\mu})^T \boldsymbol{\Sigma}^{-1} (\mathbf{A} \mathbf{z} + \mathbf{b}-\boldsymbol{\mu}))\\
       &= \frac{1}{Z'} \exp(- \frac{1}{2} (\mathbf{z} - \mathbf{A}^{-1}(\boldsymbol{\mu} - \mathbf{b}))^T (\mathbf{A}^{-1} \boldsymbol{\Sigma} (\mathbf{A}^{-1})^{T})^{-1} (\mathbf{z} - \mathbf{A}^{-1}(\boldsymbol{\mu} - \mathbf{b}))).
\end{align*}$$

<br>

#### 추가적인 선형 대수 기법
- 음의 고유값을 가지는 수리적 문제로 인하여, 만약 $\Sigma$가 양의 정부호 행렬이 아닐 때,

  $\mathbf{\Sigma} + \epsilon \mathbf{I}_d$와 $\epsilon > | \lambda_{\text{min}} |$를 통하여 해결 가능  

<br>

## 02-04. 수치적 적분
- 베이지안 통계는 많은 통계적 질문에 답할 수 있는 이론적으로 잘 정립된 프레임워크를 제공
  
  하지만 답은 분석적으로 풀 수 없는 복잡한 적분의 평가에 따라 달라지기 때문에 실제로 적용하기 어려운 경우가 존재

<br>

- 수치적 적분은 유한한 합을 통해 정적분을 근사하는 것과 관련
$$\int_V f(x) \mathrm{d}x \approx \sum_i w_i f(x_i).$$


- 공간이 1차원이고, V가 유계이며, $f(x)$가 참한 함수(Well-behaved)일 때, 이 문제를 해결하기 위한 여러 효율적인 방법이 존재

- 수치 적분을 위한 가장 간단한 방법은 균일한 그리드를 기반으로 하며,

  그리드 지점 사이의 함수를 보간하는 다양한 방법은 서로 다른 가중치 $w_i$를 가진 서로 다른 알고리즘을 생성

  1. 상수 보간법 (Piece-wise constant) - [정사각형 규칙 (rectangle rule)](https://en.wikipedia.org/wiki/Riemann_sum)
  2. 선형 보간법 (Piece-wise linear) - [사다리꼴 규칙 ( trapezoidal rule)](https://ko.wikipedia.org/wiki/%EC%82%AC%EB%8B%A4%EB%A6%AC%EA%BC%B4_%EA%B3%B5%EC%8B%9D)

  3. 이차 보간법 (Piece-wise quadratic) - [심슨 규칙 (Simpson's rule)](https://ko.wikipedia.org/wiki/%EC%8B%AC%ED%94%84%EC%8A%A8_%EA%B3%B5%EC%8B%9D)


- 이러한 간단한 구적법은 1차원에서는 잘 작동하지만, 공간의 차원이 커지면 현실적으로 불가능

- 이러한 고차원 수치 적분을 위한 효율적인 방법은 알려져 있지 않으나,

  몬테카를로 방법이 최선의 선택이 될 수 있음

<br>

## 적분 소프트웨어
- 수치 소프트웨어 패키지에는 일반적으로 프로세스를 훨씬 더 효율적으로 만들기 위해 여러 고급 트릭을 적용하는 적분 계산 기능이 포함

- Python에서 표준 1D 수치 적분은 `scipy.integrate.quad`에 의해 수행
  - 이 함수는 평가 지점을 선택하기 위해 적응형 방법을 사용하므로 대상 함수에 직접 접근해야 합니다. `scipy.integrate` 모듈에는 고차원 적분을 위한 함수도 존재

- 때로는 적분을 전체 함수로 쉽게 작성할 수 없고 고정된 점 집합에서만 평가할 수 있음
  - Python에서 함수 `scipy.integrate.trapz`와 `scipy.integrate.simps`는 이러한 고정된 함수 평가 집합에서 적분을 계산
- 다른 수치 소프트웨어에도 유사한 기능이 존재